In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import joblib
import requests
import json

In [39]:
import pyodbc
# Connection string for SQL Server
conn_str = (
    "DRIVER={SQL Server};"
    "SERVER=DESKTOP-37PS163;"
    "DATABASE=AB;"
   # "UID=your_username;"
    #"PWD=your_password"
)

# Connect to the SQL Server database
conn = pyodbc.connect(conn_str)

# Read the SQL table into a DataFrame
df = pd.read_sql_query("SELECT * from delivery_data", conn)

# Display the DataFrame
print(df)

# Close the connection
conn.close()


    customer_id        location_lat        location_long order_priority  \
0          4174   37.55158787254407  -121.67760658118294           High   
1          4507     37.164834261137  -122.06913790846197            Low   
2          1860  37.411255116585366  -121.60328571015988         Medium   
3          2294   37.77760228373523  -122.42871972994483         Medium   
4          2130  37.480370081866354  -121.54533996996669           High   
..          ...                 ...                  ...            ...   
995        3497   37.39436532381835  -122.20156508763813         Medium   
996        1009   37.54941423843045  -122.25961761130051            Low   
997        3871  37.746875716135825  -122.18810091999572         Medium   
998        1797   37.71549090709927  -122.24395243489505         Medium   
999        4313   37.52123413248533  -122.46920995033955         Medium   

      package_weight_kg        delivery_time vehicle_id traffic_level  \
0     33.79612184893208  2

C:\Users\Kuldip\AppData\Local\Temp\ipykernel_23248\467432642.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * from delivery_data", conn)


In [40]:
from sklearn.metrics import mean_squared_error

# Convert delivery_time to datetime
df['delivery_time'] = pd.to_datetime(df['delivery_time'])

# Feature Engineering: Extract time-based features
df['hour_of_day'] = df['delivery_time'].dt.hour
df['day_of_week'] = df['delivery_time'].dt.dayofweek

# Define target variable
target = 'historical_traffic_delay_min'
features = ['package_weight_kg', 'traffic_level', 'temperature_C', 
            'precipitation_mm', 'wind_speed_mps', 'hour_of_day', 'day_of_week']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Train Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate Model
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred) # type: ignore
print(f"MAE: {mae}, MSE: {mse}")

# Save Model
import joblib
joblib.dump(model, "delivery_time_predictor.pkl")

# API Integration: Fetch real-time traffic data (Google Maps API example)
def get_google_maps_traffic(origin, destination, api_key):
    url = f"https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    return data

# API Integration: Fetch real-time weather data (OpenWeather API example)
def get_weather_data(lat, lon, api_key):
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"
    response = requests.get(url)
    data = response.json()
    return data

# Example API usage (Replace with actual API keys)
GOOGLE_API_KEY = "your_google_maps_api_key"
OPENWEATHER_API_KEY = "your_openweather_api_key"
traffic_data = get_google_maps_traffic("37.7749,-122.4194", "37.3382,-121.8863", GOOGLE_API_KEY)
weather_data = get_weather_data(37.7749, -122.4194, OPENWEATHER_API_KEY)

print("Traffic Data Sample:", json.dumps(traffic_data, indent=2))
print("Weather Data Sample:", json.dumps(weather_data, indent=2))


MAE: 11.21466929357355, MSE: 175.40625317570164
Traffic Data Sample: {
  "error_message": "The provided API key is invalid. ",
  "routes": [],
  "status": "REQUEST_DENIED"
}
Weather Data Sample: {
  "cod": 401,
  "message": "Invalid API key. Please see https://openweathermap.org/faq#error401 for more info."
}
